### This notebook uses pandas and jupyter widgets to explore data created from LTER site metadata using metadataEvaluation.py
* Check the occurrence of metadata elements and compare percentages across time-stamped collections

* Create lists of records that do not contain an element

* Create lists of the content that occurs at a selected element 

* count the unique values of content at a selected element to see what variations occurr at selected elements. Useful for identifiying opportunities to ensure consistency. Also useful to see if nonstandard element content has a standardized location

In [ ]:
import glob
from contextlib import contextmanager
import os
import pandas as pd

#2005_EML_Evaluated.csv.gz
#2005_EML_Occurrence.csv
#2005_EML_RAD.csv
#2005_EML_QuickE.csv
#2005_EML_XPathOccurrence.csv
#2005_EML_XPathCounts.csv
@contextmanager
def cd(newdir):
    prevdir = os.getcwd()
    os.chdir(os.path.expanduser(newdir))
    try:
        yield
    finally:
        os.chdir(prevdir)

for subdir, dirs, files in os.walk('../data'):
    for dir in dirs:
        with cd('../data/'+dir):
            XPathCountsList=glob.glob('*_EML_XpathCounts.csv')
            mystring='../../data/'+dir+'/'
            DataDestination='../../CombinedData/'+dir+'_XPathCounts.csv'
            XPathCountsList.sort()
            print(XPathCountsList)

In [ ]:
import glob
from contextlib import contextmanager
import os
import pandas as pd

#2005_EML_Evaluated.csv.gz
#2005_EML_Occurrence.csv
#2005_EML_RAD.csv
#2005_EML_QuickE.csv
#2005_EML_XPathOccurrence.csv
#2005_EML_XPathCounts.csv
@contextmanager
def cd(newdir):
    prevdir = os.getcwd()
    os.chdir(os.path.expanduser(newdir))
    try:
        yield
    finally:
        os.chdir(prevdir)
for subdir, dirs, files in os.walk('../data'):
    for dir in dirs:
        with cd('../data/'+dir):
            print(os.getcwd())
            EvaluatedList=glob.glob('*_EML_Evaluated.csv.gz')
EvaluatedList            

In [ ]:
tifCounter = 0
for root, dirs, files in os.walk('../metadata'):
    for file in files:    
        if file.endswith('.xml'):
            tifCounter += 1
tifCounter            

In [ ]:
import glob
from contextlib import contextmanager
import os
import pandas as pd

#2005_EML_Evaluated.csv.gz
#2005_EML_Occurrence.csv
#2005_EML_RAD.csv
#2005_EML_QuickE.csv
#2005_EML_XPathOccurrence.csv
#2005_EML_XPathCounts.csv
@contextmanager
def cd(newdir):
    prevdir = os.getcwd()
    os.chdir(os.path.expanduser(newdir))
    try:
        yield
    finally:
        os.chdir(prevdir)

def CombineConceptOccurrence(CollectionComparisons, DataDestination):
    CombinedDF = pd.concat((pd.read_csv(f) for f in CollectionComparisons)) 
    #CombinedDF.to_csv(DataDestination, mode = 'w', index=False)
    CombinedPivotDF = CombinedDF.pivot(index='Concept', columns='Collection', values='CollectionOccurrence%')
    pd.options.display.float_format = '{:,.0f}'.format
    ConceptCountsDF=CombinedPivotDF.fillna(0)
    ConceptCountsDF.columns.names = ['']
    ConceptCountsDF=ConceptCountsDF.reset_index()

    ConceptCountsDF.to_csv(DataDestination, mode = 'w', index=False)
    return ConceptCountsDF
#Using concept occurrence data products, combine them and produce a record count table with collections for columns and concepts for rows
def CombineConceptCounts(CollectionComparisons, DataDestination):
    CombinedDF = pd.concat((pd.read_csv(f) for f in CollectionComparisons))
    RecordCountCombinedPivotDF = CombinedDF.pivot(index='Concept', columns='Collection', values='RecordCount')
    pd.options.display.float_format = '{:,.0f}'.format
    RecordCountCombinedPivotDF=RecordCountCombinedPivotDF.fillna(0)
    RecordCountCombinedPivotDF.columns.names = ['']
    RecordCountCombinedPivotDF=RecordCountCombinedPivotDF.reset_index()
    RecordCountCombinedPivotDF.to_csv(DataDestination, mode = 'w', index=False)
    return RecordCountCombinedPivotDF

#Using xpath occurrence data products, combine them and produce a collection occurrence% table with collections for columns and concepts for rows
def CombineXPathOccurrence(CollectionComparisons, DataDestination):
    
    CombinedDF = pd.concat((pd.read_csv(f) for f in CollectionComparisons)) 
    #CombinedDF.to_csv(DataDestination, mode = 'w', index=False)
    CombinedPivotDF = CombinedDF.pivot(index='XPath', columns='Collection', values='CollectionOccurrence%')
    pd.options.display.float_format = '{:,.0f}'.format
    ConceptCountsDF=CombinedPivotDF.fillna(0)
    ConceptCountsDF.columns.names = ['']
    ConceptCountsDF=ConceptCountsDF.reset_index()

    ConceptCountsDF.to_csv(DataDestination, mode = 'w', index=False)
    return ConceptCountsDF
#Using xpath occurrence data products, combine them and produce a record count table with collections for columns and concepts for rows
def CombineXPathCounts(CollectionComparisons, DataDestination):
    #os.makedirs('../data/Combined', exist_ok=True)
    XPathCountCombinedDF = pd.concat((pd.read_csv(f) for f in CollectionComparisons), axis=0, ignore_index=True)
    XPathCountCombinedDF=XPathCountCombinedDF.fillna(0)
    XPathCountCombinedDF.columns.names = ['']

    # get a list of columns
    cols = list(XPathCountCombinedDF)
    
    # move the column to head of list using index, pop and insert
    cols.insert(0, cols.pop(cols.index('Record')))
    # use ix to reorder
    CombinedXPathCountsDF = XPathCountCombinedDF.loc[:, cols]
    cols2 = list(CombinedXPathCountsDF)
    # move the column to head of list using index, pop and insert
    cols2.insert(0, cols2.pop(cols.index('Collection')))
    # use ix to reorder
    CombinedXPathCountsDF = CombinedXPathCountsDF.loc[:, cols2]
    CombinedXPathCountsDF

    CombinedXPathCountsDF.to_csv(DataDestination, mode = 'w', index=False)
    return CombinedXPathCountsDF
 #Using xpath occurrence data products, combine them and produce a collection occurrence% table with collections for columns and concepts for rows
def CombineEvaluatedMetadata(CollectionComparisons, DataDestination):
    #os.makedirs('../data/Combined', exist_ok=True)
    CombinedDF = pd.concat((pd.read_csv(f) for f in CollectionComparisons)) 
   
    CombinedDF.to_csv(DataDestination, mode = 'w',compression='gzip', index=False)
    return CombinedDF   
        
for subdir, dirs, files in os.walk('../data'):
    for dir in dirs:
        with cd('../data/'+dir):
            #print(os.getcwd())
            EvaluatedList=glob.glob('*_EML_Evaluated.csv.gz')
            mystring='../../data/'+dir+'/'
            DataDestination='../../CombinedData/'+dir+'_EvaluatedContent.csv.gz'
            EvaluatedList.sort()
            EvaluatedList=[mystring + s for s in EvaluatedList]
            CombineEvaluatedMetadata(EvaluatedList, DataDestination)

            OccurrenceList=glob.glob('*_EML_Occurrence.csv')
            mystring='../../data/'+dir+'/'
            DataDestination='../../CombinedData/'+dir+'_ConceptOccurrence.csv'
            OccurrenceList.sort()
            OccurrenceList=[mystring + s for s in OccurrenceList]
            CombineConceptOccurrence(OccurrenceList, DataDestination)
            
            #RADList=glob.glob('*_EML_RAD.csv')
            #mystring='../../data/'+dir+'/'
            #DataDestination='../../CombinedData/'+dir+'_ConceptCounts.csv'
            #RADList.sort()
            #RADList=[mystring + s for s in RADList]
            #CombineConceptCounts(RADList, DataDestination)
            
            #QuickEList=glob.glob('*_EML_QuickE.csv')
            #mystring='../../data/'+dir+'/'
            #DataDestination='../../CombinedData/'+dir+'_QuickE.csv'
            #QuickEList.sort()
            #QuickEList=[mystring + s for s in QuickEList]
            #CombineXPathCounts(QuickEList, DataDestination)
            
            XPathCountsList=glob.glob('*_XpathCounts.csv')
            mystring='../../data/'+dir+'/'
            DataDestination='../../CombinedData/'+dir+'_XpathCounts.csv'
            XPathCountsList.sort()
            XPathCountsList=[mystring + s for s in XPathCountsList]
            CombineXPathCounts(XPathCountsList, DataDestination)
            
            XPathOccurrenceList=glob.glob('*_EML_XPathOccurrence.csv')
            mystring='../../data/'+dir+'/'
            DataDestination='../../CombinedData/'+dir+'_XPathOccurrence.csv'
            XPathOccurrenceList.sort()
            XPathOccurrenceList=[mystring + s for s in XPathOccurrenceList]
            CombineXPathOccurrence(XPathOccurrenceList, DataDestination)




In [ ]:
EvaluatedList

##### Combine XPath Occurrence csv from each lab to create an overview of the repository at different points in time.


In [ ]:
import pandas as pd
pd.set_option('max_colwidth',400)
pd.set_option('max_rows',400)
CombinedXPathOccurrenceDF=pd.read_csv('../data/Combined/Re3dataXPathOccurrence.csv')
CombinedXPathOccurrenceDF=CombinedXPathOccurrenceDF[:-1]
CombinedXPathOccurrenceDF

Create a dataframe of the xpath counts for each record and collection date.

In [ ]:
CombinedXPathCountsDF=pd.read_csv('../data/Combined/Re3dataXPathCounts.csv')
CombinedXPathCountsDF

Create a dataframe of the records in the collection missing any element used by at least one record in the collection, selectable by dropdown widget

Example: /@schemaLocation provides a list of the records that do not have any content at the schemaLocation attribute of the root element

In [ ]:
from ipywidgets import *
import ipywidgets as widgets
from IPython.display import display
XPathChoices=list(CombinedXPathCountsDF)
XPathChoices=XPathChoices[2:]
def RecordsMissingXPath(XPathChoice):
    global xPathChoice
    global doesntHaveEssentialDF
    xPathChoice=XPathChoice
    doesntHaveDF=CombinedXPathCountsDF.loc[CombinedXPathCountsDF[xPathChoice] == 0.0]
    doesntHaveEssentialDF=doesntHaveDF[['Collection','Record']]
    return doesntHaveEssentialDF
interact(RecordsMissingXPath, XPathChoice=XPathChoices) 

Determine distribution of content at a chosen element, observe opportunities to improve consistency. Load one of the collections if the combined data is from the same organization/collection date. 

Example: /@schemaLocation lists all the records that do have content at the schemaLocation attribute of the root element

In [ ]:
DataFiles=[]
for dirpath, dirnames, filenames in os.walk("../data"):
    for filename in [f for f in filenames if f.endswith("Evaluated.csv.gz")]:
        DataFiles.extend([os.path.join(dirpath, filename)])
def EvaluatedDataChoice(CSVchoice):
    global EvaluatedMetadataDF
    EvaluatedMetadataDF=pd.read_csv(CSVchoice)
    return CSVchoice
interact(EvaluatedDataChoice, CSVchoice=DataFiles)
    




#### Now select an element's xpath to see a distribution of the content in the collection

In [ ]:


def ContentAtElement(XPathChoice):
    global ContentAtElementDF
    #xPathChoice=XPathChoice
    ContentAtElementDF=EvaluatedMetadataDF.loc[EvaluatedMetadataDF.XPath==XPathChoice]
    ContentAtElementDF=ContentAtElementDF[['Record','Content']]
    distribution=ContentAtElementDF.groupby('Content').size().sort_values(0,False)

    return XPathChoice, distribution 

interact(ContentAtElement, XPathChoice=XPathChoices)

Look at the content of a specific element. Investigate if unique content should be used as bright points for other labs or if there are opportunities to improve element choice for content

In [ ]:
XPathChoices2=list(CombinedXPathCountsDF)
XPathChoices2=XPathChoices2[2:]
def ContentAtElement(XPathChoice2):
    global CombinedContentAtElementDF2
    
    CombinedContentAtElementDF2=EvaluatedMetadataDF.loc[EvaluatedMetadataDF.XPath==XPathChoice2]
    CombinedContentAtElementDF2=CombinedContentAtElementDF2[['Collection','Record','Content']]

    return CombinedContentAtElementDF2

w=interact(ContentAtElement, XPathChoice2=XPathChoices2)
display(w)